In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import os

In [77]:
bhp = pd.read_csv(
'/Users/rprocious/Waterlevels_CBI/CBI-2/Rep_data/BobHallPier-BN/bobhal-unified.csv',
parse_dates=[0],
na_values=[-999, -99, 99, 'NA', 'RM'],
engine='python'   
)
bhp.columns = ['station #','date', 'pwl','station id','station_tag']


In [78]:

directory_path = r'/Users/rprocious/Waterlevels_CBI/CBI-2/data 2/NOAA/bobHallPier'
dataframes = []

# Loop through the files in the directory
for filename in sorted(os.listdir(directory_path)):
    if filename.endswith('level.csv'):

        file_path = os.path.join(directory_path, filename)
        
        
        df = pd.read_csv(file_path)
        
        dataframes.append(df)

bhp_NOAA = pd.concat(dataframes, ignore_index=True)
bhp_NOAA.drop(labels=range(0,3), axis=0, inplace=True)
bhp_NOAA.drop(labels=range(2304169,2304171), axis=0, inplace=True)

#bhp.drop(labels=range(0,467717),axis=0,inplace=True)
#bhp.reset_index(drop=True,inplace=True)
#bhp_NOAA.reset_index(drop=True,inplace=True)
bhp['date'] = pd.to_datetime(bhp['date'])
bhp_NOAA['date'] = pd.to_datetime(bhp_NOAA['Date Time'])

bhp = bhp.merge(bhp_NOAA,on='date',how='outer')


In [79]:

def date_index_locater(start_date,end_date,df):
    indices = []
# Filter rows by date range
    date_range_filter = (df['date'] >= start_date) & (df['date'] <= end_date)

# Get the indices of the rows within the date range
    indices = (df[date_range_filter].index.tolist())

    return(indices)



In [80]:
date_index_locater('2007-08-20 17:54','2007-08-20 18:54',bhp)

[1019939,
 1019940,
 1019941,
 1019942,
 1019943,
 1019944,
 1019945,
 1019946,
 1019947,
 1019948,
 1019949]

In [81]:
bhp.keys()

Index(['station #', 'date', 'pwl', 'station id', 'station_tag', 'Date Time',
       ' Water Level', ' Sigma', ' O or I (for verified)', ' F', ' R', ' L',
       ' Quality '],
      dtype='object')

In [82]:
bhp.drop(labels=range(0,1019938),axis=0,inplace=True)
bhp = bhp.drop(columns=['station #','station id','station_tag','Date Time', ' Sigma',' O or I (for verified)',' F',' R',' L',' Quality '])


In [84]:
bhp['offset'] = bhp['pwl'] - bhp[' Water Level']
df = pd.DataFrame({
    'values': bhp['offset'],
    'date': bhp['date']
})

# Convert column to numpy array
offset_list = df['values'].to_numpy()
date_list = df['date'].to_numpy()  # Extract dates as numpy array

def consecutive_runs_info(arr):
    if len(arr) == 0:
        return []

    runs = []
    
    def same_value(a, b):
        return (np.isnan(a) and np.isnan(b)) or (a == b)

    start_idx = 0
    current_val = arr[0]

    for i in range(1, len(arr)):
        if not same_value(arr[i], current_val):
            runs.append((start_idx, i, current_val, i - start_idx))
            start_idx = i
            current_val = arr[i]

    runs.append((start_idx, len(arr), current_val, len(arr) - start_idx))
    return runs

arr = offset_list
arr_rounded = np.round(arr, 3)  # Round values for stability
info = consecutive_runs_info(arr_rounded)

# Filter results and add start/end dates
filtered_info = []
for (start_idx, end_idx, val, length) in info:
    if not np.isnan(val) and val != 0 and length >= 10:
        start_date = date_list[start_idx]  # Get start date
        end_date = date_list[end_idx - 1]  # Get end date (exclusive index)
        filtered_info.append((start_idx, end_idx, val, length, start_date, end_date))

# Convert to DataFrame and copy to clipboard
df_filtered = pd.DataFrame(filtered_info, columns=['Start Index', 'End Index', 'Value', 'Length', 'Start Date', 'End Date'])
df_filtered.to_clipboard(index=False)

print(df_filtered)


    Start Index  End Index  Value  Length          Start Date  \
0        403190     403227 -0.005      37 2012-03-26 16:48:00   
1        403227     403464  0.002     237 2012-03-26 20:30:00   
2        403465     403478 -0.005      13 2012-03-27 20:18:00   
3        403488     405177 -0.005    1689 2012-03-27 22:36:00   
4        405179     406112 -0.005     933 2012-04-03 23:42:00   
5        406114     406182 -0.005      68 2012-04-07 21:12:00   
6        406183     408030 -0.005    1847 2012-04-08 04:06:00   
7        408031     408976 -0.005     945 2012-04-15 20:54:00   
8        408977     409642 -0.005     665 2012-04-19 19:30:00   
9        409643     410111 -0.005     468 2012-04-22 14:06:00   
10       480757     481420  0.001     663 2013-02-12 21:30:00   
11       481422     482624  0.001    1202 2013-02-15 16:00:00   
12       482630     489644  0.001    7014 2013-02-20 16:48:00   
13       489645     489798  0.001     153 2013-03-21 22:18:00   
14       571233     57142

In [85]:
df_filtered.to_clipboard()

In [86]:
bhp[403460:403485]

,date,pwl,Water Level,offset
1423398,2012-03-27 19:48:00,6.821,6.819,0.002
1423399,2012-03-27 19:54:00,6.806,6.804,0.002
1423400,2012-03-27 20:00:00,6.803,6.801,0.002
1423401,2012-03-27 20:06:00,6.797,6.795,0.002
1423402,2012-03-27 20:12:00,6.781,6.795,-0.014
1423403,2012-03-27 20:18:00,6.789,6.794,-0.005
1423404,2012-03-27 20:24:00,6.783,6.788,-0.005
1423405,2012-03-27 20:30:00,6.781,6.786,-0.005
1423406,2012-03-27 20:36:00,6.780,6.785,-0.005
1423407,2012-03-27 20:42:00,6.774,6.779,-0.005


In [96]:
p21 = pd.read_csv(
'/Users/rprocious/Waterlevels_CBI/CBI-2/Rep_data/PortIsabel-BN/ptisab-unified.csv',
parse_dates=[0],
na_values=[-999, -99, 99, 'NA', 'RM'],
engine='python'   
)
p21.columns = ['station #','date', 'pwl','station id','station_tag']



directory_path = r'/Users/rprocious/Waterlevels_CBI/CBI-2/data 2/NOAA/portIsabel'
dataframes = []

# Loop through the files in the directory
for filename in sorted(os.listdir(directory_path)):
    if filename.endswith('level.csv'):

        file_path = os.path.join(directory_path, filename)
        
        
        df = pd.read_csv(file_path)
        
        dataframes.append(df)

p21_NOAA = pd.concat(dataframes, ignore_index=True)
p21_NOAA.drop(labels=range(0,3), axis = 0,inplace=True)
p21_NOAA.reset_index(drop = True, inplace=True)

p21['date'] = pd.to_datetime(p21['date'])
p21_NOAA['date'] = pd.to_datetime(p21_NOAA['Date Time'],errors='coerce')

p21 = p21.merge(p21_NOAA,on='date',how='outer')
p21 = p21.drop(columns=['station #','station id','station_tag','Date Time', ' Sigma',' O or I (for verified)',' F',' R',' L',' Quality '])

p21['offset'] = p21['pwl'] - p21[' Water Level']
df = pd.DataFrame({
    'values': p21['offset'],
    'date': p21['date']
})

# Convert column to numpy array
offset_list = df['values'].to_numpy()
date_list = df['date'].to_numpy()  # Extract dates as numpy array

def consecutive_runs_info(arr):
    if len(arr) == 0:
        return []

    runs = []
    
    def same_value(a, b):
        return (np.isnan(a) and np.isnan(b)) or (a == b)

    start_idx = 0
    current_val = arr[0]

    for i in range(1, len(arr)):
        if not same_value(arr[i], current_val):
            runs.append((start_idx, i, current_val, i - start_idx))
            start_idx = i
            current_val = arr[i]

    runs.append((start_idx, len(arr), current_val, len(arr) - start_idx))
    return runs

arr = offset_list
arr_rounded = np.round(arr, 3)  # Round values for stability
info = consecutive_runs_info(arr_rounded)

# Filter results and add start/end dates
filtered_info = []
for (start_idx, end_idx, val, length) in info:
    if not np.isnan(val) and val != 0 and length >= 10:
        start_date = date_list[start_idx]  # Get start date
        end_date = date_list[end_idx - 1]  # Get end date (exclusive index)
        filtered_info.append((start_idx, end_idx, val, length, start_date, end_date))

# Convert to DataFrame and copy to clipboard
df_filtered = pd.DataFrame(filtered_info, columns=['Start Index', 'End Index', 'Value', 'Length', 'Start Date', 'End Date'])
df_filtered.to_clipboard(index=False)

print(df_filtered)


     Start Index  End Index  Value  Length          Start Date  \
0        1384995    1385170  0.121     175 2011-10-19 19:30:00   
1        1385227    1385274  0.001      47 2011-10-20 18:42:00   
2        1385276    1385722  0.001     446 2011-10-20 23:36:00   
3        1385723    1385739  0.001      16 2011-10-22 20:18:00   
4        1385740    1386460  0.001     720 2011-10-22 22:00:00   
..           ...        ...    ...     ...                 ...   
374      2328901    2329101 -0.002     200 2022-07-26 18:06:00   
375      2386733    2386747  0.004      14 2023-03-24 17:18:00   
376      2386752    2389588  0.004    2836 2023-03-24 19:12:00   
377      2389591    2389632  0.004      41 2023-04-05 15:06:00   
378      2389635    2389819  0.004     184 2023-04-05 19:30:00   

               End Date  
0   2011-10-20 12:54:00  
1   2011-10-20 23:18:00  
2   2011-10-22 20:06:00  
3   2011-10-22 21:48:00  
4   2011-10-25 21:54:00  
..                  ...  
374 2022-07-27 14:00:00  

In [94]:
p21_NOAA.keys()

Index(['Date Time', ' Water Level', ' Sigma', ' O or I (for verified)', ' F',
       ' R', ' L', ' Quality '],
      dtype='object')